In [1]:

# coding: utf-8

#TODO other code impovments

# In[1]:


import pygame
from pygame.locals import *
import random
import math
import string


#Sources
#http://usingpython.com/pygame-intro/
#https://pythonprogramming.net/pygame-python-3-part-1-intro/

#Global things
clock = pygame.time.Clock()
wait=False
agent_list =[]


# In[2]:

#Constants Box

AGENT_RADIUS =20
AGENT_SPEED=3
AGENT_COLOR=(100,100,100)

width= 500
height =500
TITLE="Title"

WHITE = (255,255,255)
BLACK = (0,0,0)
red = (255,0,0)
green =(0,255,0)
lime_green=(50,205,50)

BUTTON_WIDTH=100
BUTTON_HEIGHT=60
BUTTON_X=20
BUTTON_Y=20
BUTTON_SPACE=10


ACCEPTED = string.digits


#http://programarcadegames.com/python_examples/f.php?file=bouncing_balls.py
class Agent:

    #Initialize variables
    def __init__(self):
        self.x =0
        self.y =0
        self.change_x =0
        self.change_y =0
        self.color = AGENT_COLOR

    def bounce(self):
        angle = random.uniform(0,2*math.pi)
        self.change_x =math.cos(angle)*AGENT_SPEED
        self.change_y =math.sin(angle)*AGENT_SPEED


def make_agent():
    agent=Agent()
    agent.x=random.randrange(AGENT_RADIUS,width-AGENT_RADIUS)
    agent.y=random.randrange(AGENT_RADIUS+(BUTTON_Y+BUTTON_HEIGHT),height-AGENT_RADIUS)

    agent.bounce()

    #make sure it's not 0
    if agent.change_x ==0 or agent.change_y ==0:
        print("oooops")
        global wait
        wait =True

    return agent


# In[3]:

def go_function(button):
    button.color=lime_green
    agent_list.append(make_agent())

def stop_function(button):
    global wait
    wait=True


class button:

    def __init__(self,x,y,color,text,font,display,function):
        self.width=100
        self.heigth=60
        self.x=x
        self.y=y
        self.color=color
        self.text=text
        self.font=font
        self.Display=display
        self.function=function

    def draw_button(self):
        pygame.draw.rect(self.Display, self.color,(self.x,self.y,self.width,self.heigth))

        #empty string = false, if there is text then draw it
        if self.text:
            #draw the text
            textSurf, textRect = text_objects(self.text, self.font)
            textRect.center = ((self.x+(self.width/2)), (self.y+(self.heigth/2)) )
            self.Display.blit(textSurf, textRect)

    def getRect(self):
        return (self.x,self.y,self.width,self.heigth)

    def execute(self):
        self.function(self)


# In[4]:
#https://github.com/Mekire/pygame-textbox
#modified from above source
class TextBox():
    def __init__(self,rect):
        self.rect = pygame.Rect(rect)
        self.buffer = [] #this holds the input
        self.final = None #this is the current input displayed
        self.text_size =10

        self.rendered = None
        self.render_rect = None
        self.render_area = None

        self.blink = True
        self.blink_timer = 0.0

        defaults = {
                    "active" : False,
                    "color" : pygame.Color("white"),
                    "font_color" : pygame.Color("black"),
                    "outline_color" : pygame.Color("black"),
                    "outline_width" : 2,
                    "active_color" : pygame.Color("blue"),
                    "font" : pygame.font.Font("freesansbold.ttf",self.text_size),
                    "clear_on_enter" : True,
                    "inactive_on_enter" : True}

        self.__dict__.update(defaults)

    def get_event(self,event):
        if event.type == pygame.KEYDOWN and self.active:
            if event.key in (pygame.K_RETURN,pygame.K_KP_ENTER):
                self.execute()
            elif event.key == pygame.K_BACKSPACE:
                if self.buffer:
                    self.buffer.pop()
            elif event.unicode in ACCEPTED:
                self.buffer.append(event.unicode)
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            self.active = self.rect.collidepoint(event.pos)

    def execute(self):
        self.active = not self.inactive_on_enter
        if self.clear_on_enter:
            for i in range(int("".join(self.buffer))):
                agent_list.append(make_agent())

            self.buffer = []

    def update(self):
        new = "".join(self.buffer)
        if new != self.final:
            self.final = new
            self.rendered = self.font.render(self.final, True, self.font_color)
            self.render_rect = self.rendered.get_rect(x=self.rect.x+2,centery=self.rect.centery)
            if self.render_rect.width > self.rect.width-6:
                offset = self.render_rect.width-(self.rect.width-6)
                self.render_area = pygame.Rect(offset,0,self.rect.width-6,self.render_rect.height)
            else:
                self.render_area = self.rendered.get_rect(topleft=(0,0))
        if pygame.time.get_ticks()-self.blink_timer > 200:
            self.blink = not self.blink
            self.blink_timer = pygame.time.get_ticks()

    def draw(self,surface):
        outline_color = self.active_color if self.active else self.outline_color
        outline = self.rect.inflate(self.outline_width*2,self.outline_width*2)
        surface.fill(outline_color,outline)
        surface.fill(self.color,self.rect)
        if self.rendered:
            surface.blit(self.rendered,self.render_rect,self.render_area)
        if self.blink and self.active:
            curse = self.render_area.copy()
            curse.topleft = self.render_rect.topleft
            surface.fill(self.font_color,(curse.right+1,curse.y,2,curse.h))
# In[5]:

def text_objects(text, font):
    textSurface = font.render(text, True, BLACK)
    return textSurface, textSurface.get_rect()

#http://cgp.wikidot.com/circle-to-circle-collision-detection
def collision(x1,y1,x2,y2):
    dx=x1-x2
    dy=y1-y2
    r=AGENT_RADIUS+AGENT_RADIUS
    if((dx*dx)+(dy*dy)<r*r):
        return True
    return False



def main():
    pygame.init()
    Display = pygame.display.set_mode((width,height))
    pygame.display.set_caption(TITLE)

    #Text Font
    smallText = pygame.font.Font("freesansbold.ttf",20)
    largeText = pygame.font.Font("freesansbold.ttf",50)

    text = TextBox((BUTTON_X+2*(BUTTON_WIDTH+BUTTON_SPACE),BUTTON_Y,BUTTON_WIDTH,BUTTON_HEIGHT))

    #Make the buttons
    go_button = button(BUTTON_X,BUTTON_Y,green,"GO!",smallText,Display,go_function)
    stop_button =button(BUTTON_X+BUTTON_WIDTH+BUTTON_SPACE,BUTTON_Y,red,"Stop!",smallText,Display,stop_function)

    #put all the buttons in a list
    button_list =[]
    button_list.append(go_button)
    button_list.append(stop_button)

    def pause():
        paused = True
        global wait
        TextSurf, TextRect = text_objects("Paused", largeText)
        TextRect.center = ((width/2),(height/2))
        Display.blit(TextSurf, TextRect)
        while paused:
            for event in pygame.event.get():

                if event.type == pygame.QUIT:
                    run=False
                    paused=False
                    wait=False
                    #pygame.quit()
                    #quit()
                elif event.type == pygame.KEYDOWN:
                    #https://www.pygame.org/docs/ref/key.html
                    if event.key == pygame.K_g:
                        wait=False
                        paused = False

            #TODO make button to unpause
            pygame.display.update()

    run =True
    global wait
    while run:
        if wait:
            pause()

        for event in pygame.event.get():
            if event.type == QUIT:
                run=False
            elif event.type == pygame.KEYDOWN:
                #Make new agents by pressing space
                if event.key == pygame.K_SPACE:
                    agent_list.append(make_agent())

                #press p to pause game
                elif event.key == pygame.K_p:
                    wait=True

                #check if input box is active
                elif text.active:
                    if event.key in (pygame.K_RETURN,pygame.K_KP_ENTER):
                        text.execute()
                    #Delete last input
                    elif event.key == pygame.K_BACKSPACE:
                        if text.buffer:
                            text.buffer.pop()
                    #if input is valid (only int are) then add it to the end
                    elif event.unicode in ACCEPTED:
                        text.buffer.append(event.unicode)
            #left mouse click
            elif event.type == pygame.MOUSEBUTTONUP and event.button==1:
                #get mouse position
                mouse = pygame.mouse.get_pos()

                text.active = text.rect.collidepoint(event.pos)

                #if the mouse is above the input box it is not above anything else
                if not text.active:
                    #check if the mouse is above a button, if it is execute that button
                    for b in button_list:
                        r = pygame.Rect(b.getRect())
                        if r.collidepoint(mouse):
                            b.execute()
                            break


        #list of dictionary of agents that have moved, resets each loop
        moved_agents =[]

        #Agent Logic
        for agent in agent_list:
            move=True
            agent.color=AGENT_COLOR
            x=agent.x+agent.change_x
            y=agent.y+agent.change_y
            #check for collision with other agents
            for m in moved_agents:
                if(collision(x,y,m['x'],m['y'])):
                    m['agent'].color=lime_green
                    agent.color=lime_green

                    #calculate how to bounce the agents
                    distance = agent.x-m['x'], agent.y-m['y']
                    norm=math.sqrt(distance[0]**2+distance[1]**2)
                    direction=distance[0]/norm,distance[1]/norm


                    agent.change_x=direction[0]*AGENT_SPEED
                    agent.change_y=direction[1]*AGENT_SPEED

                    m['agent'].change_x=agent.change_x*-1
                    m['agent'].change_y=agent.change_y*-1

                    move=False

            moved_agents.append({'x':agent.x,'y':agent.y,'agent':agent})

            if(move):
                # Move the center
                agent.x += agent.change_x
                agent.y += agent.change_y

                #Stay in bounds
                if agent.y > height - AGENT_RADIUS or agent.y < AGENT_RADIUS+(BUTTON_Y+BUTTON_HEIGHT):
                    agent.change_y *= -1
                    agent.y+=agent.change_y

                if agent.x > width - (AGENT_RADIUS+BUTTON_Y) or agent.x < AGENT_RADIUS:
                    agent.change_x *= -1
                    agent.x+=agent.change_x

        #Make the display white then draw everything
        Display.fill(WHITE)

        # Draw the agents
        for agent in agent_list:
            #the x and y cordinates are kept as floats, but to draw they need to be int
            pygame.draw.circle(Display, agent.color, [int(round(agent.x)), int(round(agent.y))], AGENT_RADIUS)

        #draw the buttons
        for b in button_list:
            b.draw_button()

        #draw the input box
        text.update()
        text.draw(Display)

        # Limit to 60 frames per second
        clock.tick(60)

        #update the screen
        pygame.display.flip()

    #end of loop we exit
    pygame.quit()
if __name__ == "__main__":
    main()


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
